# Using `OpenAILLM`

In this notebook, we show how to use the `OpenAILLM` class.

__Requirements:__

1. Have an OpenAI API key configured (for example, via the `OPENAI_API_KEY` environment variable). You can create and manage keys in the [OpenAI dashboard](https://platform.openai.com/account/api-keys).
2. Ensure this environment has network access to the OpenAI API endpoint.

In [ ]:
# install dependencies
!pip install "llm-agents-from-scratch[openai]" -q

## Instantiating an `OpenAILLM` object

In [1]:
from llm_agents_from_scratch.llms.openai import OpenAILLM

In [2]:
llm = OpenAILLM(
    model="gpt-5",
)

## Complete

We use the `.complete()` method to perform text completion with our `OpenAILLM` object. 

In [3]:
response = await llm.complete(prompt="Tell me a joke.")

In [4]:
type(response)

llm_agents_from_scratch.data_structures.llm.CompleteResult

In [5]:
print(response.response)

Why did the scarecrow get promoted? He was outstanding in his field.

Want another?


## Chat

We use the `.chat()` method to chat with `OpenAILLM` object, using the chat API.

In [6]:
user_message, response_message = await llm.chat("Tell me a joke.")

In [7]:
type(user_message)

llm_agents_from_scratch.data_structures.llm.ChatMessage

In [8]:
print(user_message)

role=<ChatRole.USER: 'user'> content='Tell me a joke.' tool_calls=None


In [9]:
type(response_message)

llm_agents_from_scratch.data_structures.llm.ChatMessage

In [10]:
print(response_message.content)

Why don’t scientists trust atoms? Because they make up everything.

Want another—punny, techy, or dad-joke style?


## Structured Output

We use the `.structured_output()` method to produce structured responses, represented as `~pydantic.BaseModel`, with our `OpenAILLM` object. Here, we'll ask it to produce a structured data class `Joke` that contains a `topic` and a `content` attribute.

In [11]:
from pydantic import BaseModel, Field


class Joke(BaseModel):
    """A structured representation of a joke."""

    topic: str = Field(description="Topic of the joke.")
    content: str = Field(description="Joke content.")

In [12]:
joke = await llm.structured_output(
    prompt="Tell me a new joke about any topic you like.",
    mdl=Joke,
)

In [13]:
joke

Joke(topic='Smart fridges', content='My smart fridge just sent me a push notification that said, "We need to talk." I opened the door and it goes, "This relationship isn\'t healthy—mostly because of what you keep putting inside me. Also, I can\'t chill until we address your leftover commitment issues."')

## Tool Calling

In this section of the notebook, we demonstrate how to perform tool calls with an `OpenAILLM`. To do so, first we'll need to create some tools using the `llm-agents-from-scratch` library.

In [14]:
from pydantic import BaseModel, ConfigDict

from llm_agents_from_scratch import (
    PydanticFunctionTool,
)

In [15]:
class AddOneParams(BaseModel):
    """Parameters for `add_one` tool."""

    model_config = ConfigDict(extra="forbid")
    x: float


def add_one(params: AddOneParams) -> int:
    """Adds one to a given number."""
    return params.x + 1

In [16]:
add_one_tool = PydanticFunctionTool(func=add_one)

The `parameters_json_schema` of a `BaseTool` object, shows how the tool's parameters will be passed to the LLM.

In [17]:
add_one_tool.parameters_json_schema

{'additionalProperties': False,
 'description': 'Parameters for `add_one` tool.',
 'properties': {'x': {'title': 'X', 'type': 'number'}},
 'required': ['x'],
 'title': 'AddOneParams',
 'type': 'object'}

### Testing the tool

NOTE: this is a direct invocation of the tool, without any LLM invocation. A `ToolCall` is how we bundle the parameters that should be passed to the tool. The `ToolCall` object is also the parameter used in the `__call__` method for all `BaseTool` types.

In [18]:
from llm_agents_from_scratch.data_structures.tool import ToolCall

In [19]:
# add one
tool_call = ToolCall(
    tool_name=add_one_tool.name,
    arguments={"x": 3.15},
)

# invoke the __call__ method
add_one_tool(tool_call)

ToolCallResult(tool_call_id='e7568a27-5e4e-410f-8984-64be30e48061', content='4.15', error=False)

In [20]:
# add one
tool_call = ToolCall(
    tool_name=add_one_tool.name,
    arguments={"x": 3.15},
)
add_one_tool(tool_call)

ToolCallResult(tool_call_id='f66dd44a-19a2-4994-9b54-81ec8e5aaadf', content='4.15', error=False)

### Get `OpenAILLM` to use the tool

In [29]:
user_message, response_message = await llm.chat(
    input="Add one to fifty-five point three. Use only the appropriate tools!",
    tools=[add_one_tool],
)

We can see that the LLM is requesting for a tool call for `add_one`.

In [30]:
response_message

ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content='', tool_calls=[ToolCall(id_='call_RP9rdXx3BncacO5CogKqKhs1', tool_name='add_one', arguments={'x': 55.3})])

In the LLM agent processing cycle, we would next perform the tool call, and pass the result back to the LLM.

In [31]:
(
    tool_messages,
    response_message,
) = await llm.continue_chat_with_tool_results(
    tool_call_results=[
        add_one_tool(
            response_message.tool_calls[0],
        ),  # returns a ToolCallResult
    ],
    chat_history=[
        user_message,
        response_message,
    ],
)

[{'type': 'message', 'content': 'Add one to fifty-five point three. Use only the appropriate tools!', 'role': 'user'}, {'type': 'function_call', 'arguments': '{"x": 55.3}', 'call_id': 'call_RP9rdXx3BncacO5CogKqKhs1', 'name': 'add_one'}, {'type': 'function_call_output', 'call_id': 'call_RP9rdXx3BncacO5CogKqKhs1', 'output': '{\n  "content": "56.3",\n  "error": false\n}'}]


In [32]:
tool_messages

[ChatMessage(role=<ChatRole.TOOL: 'tool'>, content='{\n    "tool_call_id": "call_RP9rdXx3BncacO5CogKqKhs1",\n    "content": "56.3",\n    "error": false\n}', tool_calls=None)]

In [33]:
print(tool_messages[0].content)

{
    "tool_call_id": "call_RP9rdXx3BncacO5CogKqKhs1",
    "content": "56.3",
    "error": false
}


In [34]:
response_message

ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content='56.3', tool_calls=[])

In [35]:
print(response_message.content)

56.3
